In [1]:
import requests
import bs4
from dotenv import load_dotenv
import os
import psycopg

load_dotenv()

True

In [2]:
dbconn = os.getenv("DBCONN")
conn = psycopg.connect(dbconn)
cur = conn.cursor()

In [7]:
cur.execute('''
    DROP TABLE testing;
''')
conn.commit()

In [3]:
cur.close()
conn.close()

In [55]:
# cur.execute('''
# CREATE TABLE IF NOT EXISTS financial_times_scaped(
#     tag VARCHAR(255),
#     link VARCHAR(255) PRIMARY KEY,
#     heading VARCHAR(255),
#     teaser VARCHAR(255),
#     date TIMESTAMP,
#     FOREIGN KEY (date) REFERENCES bitcoin_api_data(date)
# );
# ''')

In [8]:
def results_for_date(date):
  headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,/;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Connection": "keep-alive",
    "DNT": "1",  # Do Not Track header
    "Upgrade-Insecure-Requests": "1",
  }
  response = requests.get(f"https://www.ft.com/search?q=bitcoin&dateFrom={date}&dateTo={date}&sort=relevance", headers=headers)
  scraped_date = []
  print(response.status_code)
  if response.status_code == 200:
    soup = bs4.BeautifulSoup(response.text, 'html.parser')
    limit = 5
    search_results = soup.select("div.o-teaser__content", limit=limit)
    for result in search_results:
      scraped = [date]
      try:
        scraped.append(result.select_one("a.o-teaser__tag").get_text())
      except:
        scraped.append("")
      try:
        scraped.append(result.select_one("a.js-teaser-heading-link").get_text())
      except:
        scraped.append("")
      try:
        scraped.append(result.select_one("a.js-teaser-heading-link")["href"])
      except:
        scraped.append("")
      try:
        scraped.append(result.select_one("p.o-teaser__standfirst > a").get_text())
      except:
        scraped.append("")
      scraped_date.append(tuple(scraped))
  return scraped_date

In [9]:
results_for_date("2024-04-03")

200


[('2024-04-03',
  'Capital markets',
  'Abolishing AT1s would be a serious mistake',
  'https://www.ft.com/content/69e5e619-2686-4ce7-95d4-94bc8c6ee213',
  'Dutch government has floated idea of ditching capital instrument that is indispensable for issuers and investors'),
 ('2024-04-03',
  'Gold',
  'Gold hits new high as investors seek hedge against stubborn inflation',
  'https://www.ft.com/content/fa1b68ab-5b4e-4574-ba4e-224dcf9cc189',
  'Price of metal climbs to record of $2,295 a troy ounce'),
 ('2024-04-03',
  'FT High-Growth Companies Asia-Pacific',
  'Australia’s Commonwealth Bank tie-in gives More Telecom a boost',
  'https://www.ft.com/content/432d226f-2144-4df3-b7db-06d7999c7b35',
  'Small Melbourne-based telecoms company is thriving with investment from the country’s largest bank'),
 ('2024-04-03',
  'Live blog from April 3 2024',
  'Markets update: South Korea leads losses in Asia as equities follow US lower',
  '/content/821f5fa6-5c9d-40ce-8a1a-d682510270b6#post-f14a1488-

In [ ]:
last_date = '2024-02-06'

cur.execute('''
SELECT date FROM bitcoin_api_data WHERE date < '%s'::date ORDER BY date DESC;
''', last_date)

In [58]:
date_times = cur.fetchall()
dates = [date[0].strftime('%Y-%m-%d') for date in date_times]

In [ ]:
print(len(dates), "remaining")

In [ ]:
# only does ∼10 days before 406
for date in dates:
  scraped = results_for_date(date)
  print(scraped)
  if len(scraped) == 0:
    break
  for item in scraped:
    print("adding item", item)
    cur.execute(
      '''
        INSERT INTO financial_times_scaped(date, tag, heading, link, teaser)
        VALUES (%s, %s, %s, %s, %s);
      ''', 
      item
    )
print("done")

In [61]:
# cur.execute(
#   '''TRUNCATE TABLE financial_times_scaped;'''
# )

In [62]:
conn.commit()
cur.close()
conn.close()